In [1]:
dpi = 300 # human eye cannot detect any difference above 300
figsize = (25.60, 25.60) # full HD is (19.20, 10.80)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit

In [3]:
@njit(parallel=True, nogil=True)
def random_collatzed(n):
  x = np.linspace(1, n, n)
  collatzed_array = np.random.choice(x, size=n)
  evens = np.mod(collatzed_array, 2) == 0
  odds = np.logical_and((np.mod(collatzed_array, 2) == 1), 
                        (np.not_equal(collatzed_array, 1)))
  collatzed_array[evens] = np.divide(collatzed_array[evens], 2)
  collatzed_array[odds] = np.divide(
      np.add(np.multiply(collatzed_array[odds], 3), 1), 
      2)
  collatzed_array = np.column_stack((np.linspace(1, n, n), collatzed_array))
  return collatzed_array

def rotate_vector(data, angle):
  theta = np.radians(angle)
  co = np.cos(theta)
  si = np.sin(theta)
  rotation_matrix = np.array(((co, -si), (si, co)))
  rotated_vector = data.dot(rotation_matrix)
  return rotated_vector

n = 5000
fixed_w = 10

collatzed_array = random_collatzed(n=n)

diffed_x = np.diff(collatzed_array[:, 0])
diffed_y = np.diff(collatzed_array[:, 1])

diffed_collatz = np.column_stack((diffed_x, diffed_y))

collatz_norm = np.linalg.norm(diffed_collatz, axis=1)
collatz_unit = np.column_stack(
    (np.divide(diffed_x, collatz_norm), np.divide(diffed_y, collatz_norm))
)

collatz_unit = np.insert(collatz_unit, 0, np.array([0, 1]), axis=0)

new_points = np.add(collatzed_array, np.multiply(fixed_w, collatz_unit))

angles = np.array([30*x for x in range(1, 13)])

In [ ]:
def butterfly_collatz(angle):
  rotated_array = np.empty(shape=new_points.shape)
  for row, i in zip(new_points, range(rotated_array.shape[0])):
    rotated_array[i, :] = rotate_vector(row, angle=angle)
  return rotated_array

rotated_arrays = np.array(['rot' + str(angle) for angle in angles])

for array, angle in zip(rotated_arrays, angles):
  prog = array+'='+f'butterfly_collatz({angle})'
  exec(prog)

t = np.linspace(0, 12*np.pi, 10000)
expr = np.subtract(
    np.subtract(np.exp(np.cos(t)), np.multiply(2, np.cos(np.multiply(4, t)))),
                   np.power(np.sin(np.divide(t, 12)), 5))

x = np.multiply(np.sin(t), expr)
y = np.multiply(np.cos(t), expr)

plt.style.use('dark_background')

fig = plt.figure(figsize=figsize, dpi=dpi)
ax1 = fig.add_subplot(111, label='1')
ax2 = fig.add_subplot(111, label='2', frame_on=False)

plt.axis('off')

ax1.plot(new_points[:, 0], new_points[:, 1], linewidth=0.05, alpha=0.5)

arrays = [rot30, rot60, rot90, rot120, rot150, rot180, rot210, rot240,
                   rot270, rot300, rot330, rot360]

for array in arrays:
  ax1.plot(array[:, 0], array[:, 1], linewidth=1, alpha=0.5)

ax2.plot(x, y, linewidth=5, alpha=0.5, color='white')

ax1.axis('off')
ax2.axis('off')

plt.savefig('butterfly.png', figsize=figsize, dpi=dpi)

In [ ]:
plt.subplots(figsize=figsize, dpi=dpi, facecolor='#006a4e', 
             subplot_kw=dict(polar=True))

for array in arrays:
  plt.plot(array[:, 0], array[:, 1], linewidth=1, alpha=0.9, color='#f42a41')

plt.axis('off')

plt.savefig('bd.png', figsize=figsize, dpi=dpi, facecolor='#006a4e')

In [ ]:
def bu_rotate(data, angle):
  rotated_array = np.empty(shape=data.shape)
  for row, i in zip(new_points, range(data.shape[0])):
    rotated_array[i, :] = rotate_vector(row, angle=angle)
  return rotated_array

collatz_data = np.empty(shape=(n, len(angles)*2))

angles = np.array([30*x for x in range(1, 13)])

for angle, i in zip(angles, range(collatz_data.shape[1])):
  collatz_data[:, i:i+2] = bu_rotate(new_points, angle=angle)

collatz_data = np.column_stack((collatz_data, new_points))

x = -10.0
y = -7.0
z = 35.0
xt = np.array([])
yt = np.array([])

dt = 0.001
sigma = 10.0
R = 28.0
b = np.divide(-8.0, 3.0)
theta = np.divide(np.multiply(3., np.pi), 4.)

for t in np.arange(0, 10, dt):
  dxdt = np.multiply(sigma, np.add(-x, y))
  dydt = np.subtract(np.subtract(np.multiply(R, x), np.multiply(x, z)), y)
  dzdt = np.add(np.multiply(b, z), np.multiply(x, y))
  x = np.add(x, np.multiply(dxdt, dt))
  y = np.add(y, np.multiply(dydt, dt))
  z = np.add(z, np.multiply(dzdt, dt))
  x_rot = np.subtract(
      np.multiply(np.cos(theta), x), 
      np.multiply(np.sin(theta), y))
  y_rot = np.add(np.multiply(np.sin(theta), x), np.multiply(np.cos(theta), y))
  xt = np.append(xt, x_rot)
  yt = np.append(yt, z)

plt.style.use('dark_background')

fig = plt.figure(figsize=figsize, dpi=dpi)
ax1 = fig.add_subplot(111, label='1')
ax2 = fig.add_subplot(111, label='2', frame_on=False)

point_blocks = np.array([0, 2, 4, 6, 8, 10])

for block in point_blocks:
  plot_array = collatz_data[:, block:block+2]
  ax1.plot(plot_array[:, 0], plot_array[:, 1], linewidth=1, alpha=0.5)

ax2.plot(xt, yt, color='white', linewidth=5, alpha=0.5)
ax1.axis('off')
ax2.axis('off')

plt.savefig('lorentz.png', figsize=figsize, dpi=dpi)